In [1]:
from network import CNetwork

In [2]:
import pandas as pd
import numpy as np

In [3]:
from utils import verifyDir

In [4]:
dataset_path = "datasetsv2/"
name_dataset = "dataset_1"
length_cut = 100
random_flag = True

In [5]:
auxiliar_path = 'auxiliar_folder/' + name_dataset   + '/'
verifyDir(auxiliar_path)

In [6]:
df_train = pd.read_csv(dataset_path + "df_train_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [7]:
df_val = pd.read_csv(dataset_path + "df_val_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [8]:
df_test = pd.read_csv(dataset_path + "df_test_" + name_dataset + "_" + str(length_cut) + "_" + str(random_flag) + ".csv")

In [9]:
df_train.head(5)

,label,text,book
0,Bram Stoker,"of it might do harm . Why , even these men mig...",The Mystery Of The Sea
1,Allan Poe,"bill , even there . However , you can send fou...",The Works Of Edgar Allan Poe4
2,Allan Poe,heard her first feeble movements in the hollow...,The Works Of Edgar Allan Poe2
3,Hector Hugh,"decade , that is the insidious leaven that wil...",When William Came
4,Charles Dickens,"place , which the wearer no sooner reached tha...",Barnaby Rudge


In [10]:
from utils.text_processing import get_word_index, get_sequences, get_common_words, remove_puntuaction

In [11]:
df_train['text'] = df_train['text'].apply(lambda t: remove_puntuaction(t))

In [12]:
df_test['text'] = df_test['text'].apply(lambda t: remove_puntuaction(t))

In [13]:
df_val['text'] = df_val['text'].apply(lambda t: remove_puntuaction(t))

In [14]:
df_train.head(5)

,label,text,book
0,Bram Stoker,of it might do harm Why even these men might...,The Mystery Of The Sea
1,Allan Poe,bill even there However you can send four d...,The Works Of Edgar Allan Poe4
2,Allan Poe,heard her first feeble movements in the hollow...,The Works Of Edgar Allan Poe2
3,Hector Hugh,decade that is the insidious leaven that will...,When William Came
4,Charles Dickens,place which the wearer no sooner reached than...,Barnaby Rudge


In [15]:
classes = list(df_train['label'])  ## or 'author'
total_classes = list(set(df_train['label']))  ## or author
print("Classes: {}".format(total_classes))
print("Total classes: {}".format(len(total_classes)))
number_books = (df_train[df_train['label'] == total_classes[0]]).shape[0]
print("Total entities for each class in train: {}".format(number_books))
dict_categories = list(set(classes))
dict_categories = {cat: index for index, cat in enumerate(dict_categories)}

Classes: ['Arthur Conan Doyle', 'Joseph Conrad', 'Mark Twain', 'Thomas Hardy', 'Hector Hugh', 'Bram Stoker', 'George Eliot', 'Allan Poe', 'Jane Austen', 'Daniel Defoe', 'Charles Darwin', 'Charles Dickens', 'Pelham Grenville']
Total classes: 13
Total entities for each class in train: 4


In [16]:
y_train = [dict_categories[y] for y in df_train["label"]]

In [17]:
y_val = [dict_categories[y] for y in df_val["label"]]

In [18]:
len(y_train)

46

In [19]:
!pip install networkx

In [20]:
word_index, index_word = get_word_index(df_train["text"])

In [21]:
word_index

{'boy': '0',
 'Yes': '1',
 'trustees': '2',
 'seen': '3',
 'whole': '4',
 'understand': '5',
 'life': '6',
 'regard': '7',
 'their': '8',
 'vanished': '9',
 'popular': '10',
 'built': '11',
 'it': '12',
 'disguise': '13',
 'but': '14',
 'want': '15',
 'important': '16',
 'some': '17',
 'not': '18',
 'gigantic': '19',
 'wrapped': '20',
 'empty': '21',
 'Eastern': '22',
 'in': '23',
 'proposed': '24',
 'outside': '25',
 'quite': '26',
 'were': '27',
 'so': '28',
 'thrown': '29',
 'left': '30',
 'dark': '31',
 'necessary': '32',
 'into': '33',
 'coast': '34',
 'Japan': '35',
 'be': '36',
 'seeming': '37',
 'free': '38',
 'made': '39',
 'chance': '40',
 'correctly': '41',
 'smiled': '42',
 'she': '43',
 'though': '44',
 'answered': '45',
 'multiplied': '46',
 'could': '47',
 'being': '48',
 'that': '49',
 'brotherly': '50',
 'leaving': '51',
 'played': '52',
 'am': '53',
 'baby': '54',
 'reports': '55',
 'after': '56',
 'having': '57',
 'taking': '58',
 'die': '59',
 'upon': '60',
 'shall'

In [22]:
len(word_index)

7132

In [23]:
def read_embeddings(fname):
    with open(fname) as f:
        f.readline()
        names = []
        data = []
        for line in f:
            line = line.strip().split(",")
            names.append(line[0])
            data.append([float(v) for v in line[1:]])
        return names, data

In [24]:
def organize_data(graph2label, embed_vnames, embed_vecs):
    N = len(graph2label)
    data = [0 for _ in range(N)]
    labels = [0 for _ in range(N)]
    for i in range(len(graph2label)):
        idx = embed_vnames[i]
        data[int(idx)] = embed_vecs[i]
        labels[int(idx)] = graph2label[idx]
    return np.array(data), np.array(labels)

In [25]:
from utils.igraph2json import igraph2json1
import os

In [26]:
def graph2vec(path, operating_system, networks, labels):
    networks_dir = path + 'dataset/'
    emb_file = path + 'features/nci1.csv'
    graph2label = {}
    try: 
        os.mkdir(in_network)
        os.mkdir(path + 'features/')
    except:
        print("Existe")
    for (i, netw), label in zip(enumerate(networks), labels):
        graph2label[str(i)] = label
        igraph2json1(netw, networks_dir + str(i) + ".json")
    if operating_system == 'linux':
        path_command = 'python ./graph2vec/src/graph2vec.py --input-path ' + networks_dir + ' --output-path ' + emb_file + ' --dimensions ' + str(1024)
    os.system(path_command)
    embed_vnames, embed_vecs = read_embeddings(emb_file)
    X, y = organize_data(graph2label, embed_vnames, embed_vecs)
    return X, y


In [27]:
def get_graph2vec_features(sequences, labels):
    network_features = pd.DataFrame()
    nets = []
    for text in sequences:
        obj = CNetwork(text, model=None, index_word=None, percentages=None, path=auxiliar_path)
        network = obj.create_network()
        nets.append(network)
    X, y = graph2vec("", "linux", nets, labels)
    print("Shapes {} {}".format(X.shape, y.shape))
    return X, y

In [28]:
X_train, y_train = get_graph2vec_features(get_sequences(df_train["text"], word_index), y_train)

Nodes: 20 - Edges: 2023
Nodes: 21 - Edges: 2064
Nodes: 21 - Edges: 2181
Nodes: 20 - Edges: 2523
Nodes: 20 - Edges: 2211
Nodes: 20 - Edges: 2311
Nodes: 25 - Edges: 1967
Nodes: 22 - Edges: 1978
Nodes: 25 - Edges: 2115
Nodes: 25 - Edges: 2361
Nodes: 14 - Edges: 1998
Nodes: 19 - Edges: 2012
Nodes: 22 - Edges: 2141
Nodes: 23 - Edges: 3393
Nodes: 17 - Edges: 2329
Nodes: 21 - Edges: 2092
Nodes: 23 - Edges: 2131
Nodes: 20 - Edges: 2141
Nodes: 19 - Edges: 2065
Nodes: 23 - Edges: 2082
Nodes: 15 - Edges: 2450
Nodes: 23 - Edges: 2172
Nodes: 24 - Edges: 2083
Nodes: 19 - Edges: 2259
Nodes: 21 - Edges: 2395
Nodes: 25 - Edges: 2120
Nodes: 25 - Edges: 2226
Nodes: 17 - Edges: 1829
Nodes: 26 - Edges: 2625
Nodes: 20 - Edges: 2238
Nodes: 19 - Edges: 1994
Nodes: 22 - Edges: 2080
Nodes: 20 - Edges: 1901
Nodes: 24 - Edges: 2647
Nodes: 20 - Edges: 2117
Nodes: 22 - Edges: 2228
Nodes: 19 - Edges: 2170
Nodes: 22 - Edges: 1985
Nodes: 22 - Edges: 2057
Nodes: 24 - Edges: 2160
Nodes: 19 - Edges: 2053
Nodes: 23 - Edge

  0%|          | 0/46 [00:00<?, ?it/s]


Feature extraction started.



100%|██████████| 46/46 [00:00<00:00, 76.42it/s]



Optimization started.

Shapes (46, 1024) (46,)


In [29]:
X_train.shape

(46, 1024)

In [30]:
X_val, y_val = get_graph2vec_features(get_sequences(df_val["text"], word_index), y_val)

Nodes: 20 - Edges: 2268
Nodes: 22 - Edges: 2239
Nodes: 17 - Edges: 2038
Nodes: 19 - Edges: 2057
Nodes: 18 - Edges: 2157
Nodes: 25 - Edges: 2142
Nodes: 19 - Edges: 2231
Nodes: 21 - Edges: 1794
Nodes: 24 - Edges: 2443
Nodes: 23 - Edges: 2064
Nodes: 22 - Edges: 2173
Nodes: 19 - Edges: 2083
Nodes: 19 - Edges: 2157
Nodes: 17 - Edges: 1809
Nodes: 22 - Edges: 1966
Nodes: 24 - Edges: 2250
Existe


  0%|          | 0/46 [00:00<?, ?it/s]


Feature extraction started.



100%|██████████| 46/46 [00:00<00:00, 61.54it/s]



Optimization started.

Shapes (16, 1024) (16,)


# Training with SVM

In [31]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC, LinearSVC

In [37]:
params = {'C':[0.01,0.1,1,10,100,1000]}

classifier = GridSearchCV(LinearSVC(), params, cv=3, scoring='f1')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_val)
score = accuracy_score(y_val, y_pred)
print(y_val)
print(y_pred)
print("Score {}".format(score))
print("Best params {}".format(classifier.best_params_))
print(classification_report(y_val, y_pred))

/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/utils/validation.py", line 63, in

/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 242, in _score
    return self._sign * self._score_func(y_true, y_pred,
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/utils/validation.py", line 63, in

/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/

[10  7 12  0  8  2  6  1 10  3 11  5 12  9 11  4]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
Score 0.0625
Best params {'C': 0.01}
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.06      1.00      0.12         1
          10       0.00      0.00      0.00         2
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00         2

    accuracy                           0.06        16
   macro avg       0.00      0.08      0.01        16
weighted avg 

/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 674, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "/home/mailaucq/Documents/book_classification/venv/lib/python3.8/